In [7]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import keras.backend as K
from generator import DataGenerator
PROJECT_NAME = "LPRnet_keras"
MODEL_NAME = "depthwise_model_randomchars_perspective_tflite"

IMAGE_SHAPE = [94,24]
CHARS = "ABCDEFGHIJKLMNPQRSTUVWXYZ0123456789" # exclude I, O
CHARS_DICT = {char:i for i, char in enumerate(CHARS)}
DECODE_DICT = {i:char for i, char in enumerate(CHARS)}
NUM_CLASS = len(CHARS)+1

In [8]:
def CTCLoss(y_true, y_pred):
    # Compute the training-time loss value
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
    label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

    loss = keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)
    
    return loss

In [9]:
class small_basic_block(keras.layers.Layer):

    def __init__(self,out_channels,name=None,**kwargs):
        super().__init__(**kwargs)
        out_div4=int(out_channels/4)
        self.main_layers = [
            keras.layers.SeparableConv2D(out_div4,(1,1),padding='same',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
            keras.layers.SeparableConv2D(out_div4,(3,1),padding='same',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
            keras.layers.SeparableConv2D(out_div4,(1,3),padding='same',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
            keras.layers.SeparableConv2D(out_channels,(1,1),padding='same',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
        ]  
    
    def call(self,input):
        x = input
        for layer in self.main_layers:
            x = layer(x)
        return x

In [10]:
#test this later

class global_context(keras.layers.Layer):
    def __init__(self,kernel_size,stride,**kwargs):
        super().__init__(**kwargs)
        self.ksize = kernel_size
        self.stride = stride

    def call(self, input):
        x = input 
        avg_pool = keras.layers.AveragePooling2D(pool_size=self.ksize,strides=self.stride,padding='same')(x)
        sq = keras.layers.Lambda(lambda x: tf.math.square(x))(avg_pool)
        sqm = keras.layers.Lambda(lambda x: tf.math.reduce_mean(x))(sq)
        out = keras.layers.Lambda(lambda x: tf.math.divide(x[0], x[1]))([avg_pool , sqm])
        #out = keras.layers.Lambda(lambda x: K.l2_normalize(x,axis=1))(avg_pool)
        return out

    def get_config(self):
        return {
            "kernel_size": self.ksize,
            "stride": self.stride,
        }

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [17]:
class LPRnet(keras.Model):
    def __init__(self, input_shape=(24,94,3), **kwargs):
        super(LPRnet, self).__init__(**kwargs)
        self.input_layer = keras.layers.Input(input_shape)
        self.cnn_layers= [
            keras.layers.SeparableConv2D(64,kernel_size = (3,3),strides=1,padding='same',name='main_conv1',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(name='BN1'),
            keras.layers.ReLU(name='RELU1'),
            keras.layers.MaxPool2D(pool_size=(3,3),strides=(1,1),name='maxpool2d_1',padding='same'),
            small_basic_block(128),
            keras.layers.MaxPool2D(pool_size=(3,3),strides=(1,2),name='maxpool2d_2',padding='same'),
            small_basic_block(256),
            small_basic_block(256),
            keras.layers.MaxPool2D(pool_size=(3,3),strides=(1,2),name='maxpool2d_3',padding='same'),
            keras.layers.Dropout(0.5),
            keras.layers.SeparableConv2D(256,(4,1),strides=1,padding='same',name='main_conv2',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
            keras.layers.Dropout(0.5),
            keras.layers.SeparableConv2D(NUM_CLASS,(1,13),padding='same',name='main_conv3',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),  
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
        ]
        self.out_layers = [
            keras.layers.SeparableConv2D(NUM_CLASS,kernel_size=(1,1),strides=(1,1),padding='same',name='conv_out',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
        ]
        self.out = self.call(self.input_layer)

    def call(self,inputs,training=False):
        x = inputs
        layer_outputs = []
        for layer in self.cnn_layers:
            x = layer(x)
            layer_outputs.append(x)
        scale1 = global_context((1,4),(1,4))(layer_outputs[0])
        scale2 = global_context((1,4),(1,4))(layer_outputs[4])
        scale3 = global_context((1,2),(1,2))(layer_outputs[6])
        scale5 = global_context((1,2),(1,2))(layer_outputs[7])
        sq = keras.layers.Lambda(lambda x: tf.math.square(x))(x)
        sqm = keras.layers.Lambda(lambda x: tf.math.reduce_mean(x))(sq)
        scale4 = keras.layers.Lambda(lambda x: tf.math.divide(x[0], x[1]))([x , sqm])
        gc_concat = keras.layers.Lambda(lambda x: tf.concat([x[0], x[1], x[2], x[3], x[4]],3))([scale1, scale2, scale3, scale5,scale4])
        for layer in self.out_layers:
            gc_concat = layer(gc_concat)
        logits = keras.layers.Lambda(lambda x: tf.math.reduce_mean(x[0],axis=1))([gc_concat])
        logits = keras.layers.Softmax()(logits)
        return logits

In [18]:
import glob
import cv2

real_images_val = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\valid\\*\\*.png')
real_images = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\test\\marty\\*\\*.png')

data = []
labels = []
val_data = []
val_labels = []

for file in real_images:
    label = file.split('\\')[-1].split('_')[0].split('-')[0]
    label = label.replace("O","0")
    image = cv2.imread(file,cv2.IMREAD_COLOR)
    image = cv2.resize(image,(94,24))/256
    data.append(image)
    labels.append([CHARS_DICT[i] for i in label.split('_')[0]])

for file in real_images_val:
    label = file.split('\\')[-1].split('_')[0].split('-')[0]
    label = label.replace("O","0")
    image = cv2.imread(file,cv2.IMREAD_COLOR)
    image = cv2.resize(image,(94,24))
    val_data.append(image)
    val_labels.append([CHARS_DICT[i] for i in label.split('_')[0]])

training_set = np.array(data,dtype=np.float32)
training_labels = np.array(labels)
ragged = tf.ragged.constant(training_labels).to_tensor()
real_dataset = tf.data.Dataset.from_tensor_slices((training_set,ragged)).batch(64)


C:\Users\carlos\AppData\Local\Temp/ipykernel_1740/2036217571.py:29: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training_labels = np.array(labels)


In [19]:
if os.path.exists(MODEL_NAME):
    model = keras.models.load_model(
        MODEL_NAME, custom_objects={"global_context": global_context, "CTCLoss": CTCLoss  }
    )
else:
    model = LPRnet()
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),loss =CTCLoss)
    model.build((1,24,94,3))

In [20]:
from gen_plates_keras import *
gen = ImageGenerator()
def generate_dataset(num = 100):
    data, labels = gen.generate_images(num)
    gen_labels = []
    for label in labels:
        gen_labels.append([CHARS_DICT[i] for i in label.split('_')[0]])
    pics =np.array(data)
    labels = np.array(labels)
    training_set = np.array(pics,dtype=np.float32)
    training_labels = np.array(gen_labels)
    ragged = tf.ragged.constant(training_labels).to_tensor()
    dataset = tf.data.Dataset.from_tensor_slices((training_set,ragged)).shuffle(640).batch(64)
    return dataset
test_dataset = generate_dataset()

C:\Users\carlos\AppData\Local\Temp/ipykernel_1740/2355684483.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training_labels = np.array(gen_labels)


In [21]:
import wandb
from wandb.keras import WandbCallback
wandb.init(project=MODEL_NAME, entity="clsandoval")
wandb.config = {
  "learning_rate": 0.001,
  "epochs": 400,
  "batch_size": 64
}

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: clsandoval (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [ ]:
generate = DataGenerator()
check = tf.keras.callbacks.ModelCheckpoint(
    './trained_models/{}'.format(MODEL_NAME),
    monitor="val_loss",
    verbose=0,
    save_best_only=False,
    save_weights_only=False,
    mode="auto",
    save_freq=500,
    options=None,
)
model.fit(test_dataset)
model.fit_generator(generator=generate,validation_data=real_dataset,validation_steps=3,epochs=11,steps_per_epoch=50,callbacks=[WandbCallback(),check])


In [ ]:

TFLITE_PATH = 'tflite_models'
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("./{}/{}.tflite".format(TFLITE_PATH,MODEL_NAME), 'wb') as f:
  f.write(tflite_model)

In [ ]:
interpreter = tf.lite.Interpreter("C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\tflite_models\\depthwise_model_randomchars_perspective_tflite.tflite")
#interpreter = tf.lite.Interpreter("C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\tflite_models\\keras_lprnet_separable.tflite")
import numpy as np
import time 
import glob
import cv2

interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
real_images = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\test\\marty\\*\\*.png')
start = time.time()
ctr = 0
for file in real_images:
    label = file.split('\\')[-1].split('_')[0].split('-')[0]
    image = cv2.imread(file)
    test_image = cv2.resize(image,(94,24))/256
    test_image = np.expand_dims(test_image,axis=0)
    test_image = test_image.astype(np.float32)
    interpreter.set_tensor(input_details[0]['index'], test_image)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    decoded = keras.backend.ctc_decode(output_data,(24,),greedy=False)
    text = ""
    for i in np.array(decoded[0]).reshape(24):
        if i >-1:
            text += DECODE_DICT[i]
    if label == text:
        ctr += 1
    print(text, " "+ label)
print(time.time()-start)
print(ctr,len(real_images))

In [11]:
MODEL_PATH = 'trained_models'
TFLITE_PATH = 'tflite_models'
model = keras.models.load_model(
    os.path.join(MODEL_PATH, MODEL_NAME), custom_objects={"global_context": global_context, "CTCLoss": CTCLoss  }
)

In [12]:
import glob
import cv2
real_images_val = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\valid\\*\\*.png')
real_images = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\LPRnet-keras\\test\\*\\*\\*.png')
ctr = 0
for file in real_images:
    label = file.split('\\')[-1].split('_')[0].split('-')[0]
    image = cv2.imread(file).astype('float32')
    test_image = cv2.resize(image,(94,24))/256
    test_image = np.expand_dims(test_image,axis=0)
    preds = model.predict(test_image) 
    decoded = tf.keras.backend.ctc_decode(preds,(24,))
    text = ""
    for i in np.array(decoded[0]).reshape(24):
        if i >-1:
            text += DECODE_DICT[i]
    if label == text:
        ctr += 1
    print(text," "+ label)
print(ctr)
print(len(real_images))

86ZSCLAA  07303
71DE38SEBAE5A  1303
13008A2  1303
566TPEAA  566TPE
E5TQA  57
5E500TGAAA  59
AQ35TJAA  AA03573
AAK7063AA  AAK7053
AAEZ4G73A  AAL4673
ATG3T7TFA  ACG3105
AD35293AA  AD35293
DAL5017AA  DAL5017
DAMX900A  DAM7900
DC8T27  DC18721
6ZGTE5AA  G2G725
NAF6171  NAF6171
NBK7446  NBK7446
MG5TE5ZAAA  NC57254
NCG1092Z  NCG1092
NCZ5306  NCZ5306
EE3253FAA  NDB3253
NET0G3GAA  NE70636
NET0G3GAA  NE84727
NEJ990D0A  NEJ9900
N6E2785AA  NH
PTY5UTAA  PYV501
VJNXTLBAAA  TJX496
V50AT54AAA  WSO154
ZUVZ5TAA  ZJV257
E305PFG5AA  1303
943N0XAA  943NOX
UDAE3011AA  DAE3011
NAH7401A  NAH7401
M8G1384A  NBG1384
NBV2323  NBV2323
NC16514A  NCI6514
NCJ9396  NCJ9396
NCU3758A  NCU3758
ND11397  NDI1397
3ZT51EDAAA  NDT5180
NDU6721AA  NDU6721
ENET179AA  NET1793
NGG6277AA  NGG6277
TTA8KQJTAA  TYX907
0GEB09AA  UGF809
BET339AA  WET339
EF19E0FAAA  XSC203
YSB054  YS6054
AAH70G2A  AAH7062
AAZ2G55A  AAZ2655
DBG5953A  ABG5953
ACZED0AA  ACZ4349
1P1BTT5A  APA8716
NAD1140  NAD1140
NLV727UJA  NBQ2703
NETE34ZA  NCY8340
NDG3354 

In [24]:
val_training_set = np.array(val_data,dtype=np.float32)
val_training_labels = np.array(val_labels)
val_ragged = tf.ragged.constant(val_training_labels).to_tensor()
val_dataset = tf.data.Dataset.from_tensor_slices((val_training_set,val_ragged)).shuffle(640).batch(64)

C:\Users\carlos\AppData\Local\Temp/ipykernel_21744/3660101917.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  val_training_labels = np.array(val_labels)
